# 如何添加节点重试策略

在许多用例中，您可能希望节点具有自定义重试策略。您可能希望这样做的一些示例是，如果您正在调用 API、查询数据库或调用 LLM 等。

为了配置重试策略，您必须将 `retryPolicy` 参数传递给 `addNode` 函数。`retryPolicy` 参数接受一个 `RetryPolicy` 命名元组对象。下面我们用默认参数实例化一个 `RetryPolicy` 对象：

In [1]:
import { RetryPolicy } from "@langchain/langgraph"

const retryPolicy: RetryPolicy = {};

如果您想了解有关每个参数的作用的更多信息，请务必阅读[参考](https://langchain-ai.github.io/langgraphjs/reference/types/langgraph.RetryPolicy.html)。

## 将重试策略传递给节点

最后，当我们调用 `addNode` 函数时，我们可以传递 `RetryPolicy` 对象。在下面的示例中，我们将两个不同的重试策略传递给每个节点：

In [4]:
import Database from "better-sqlite3"
import { ChatAnthropic } from "@langchain/anthropic"
import { MessagesAnnotation, StateGraph, START, END } from "@langchain/langgraph"
import { AIMessage } from "@langchain/core/messages"

// 创建内存数据库
const db: typeof Database.prototype = new Database(':memory:');

const model = new ChatAnthropic({ model: "claude-3-5-sonnet-20240620" });

const callModel = async (state: typeof MessagesAnnotation.State) => {
    const response = await model.invoke(state.messages);
    return { messages: [response] };
}

const queryDatabase = async (state: typeof MessagesAnnotation.State) => {
    const queryResult: string = JSON.stringify(db.prepare("SELECT * FROM Artist LIMIT 10;").all());

    return { messages: [new AIMessage({content: "queryResult"})]};
};

const workflow = new StateGraph(MessagesAnnotation)
    // 定义我们将在其之间循环的两个节点
    .addNode("call_model", callModel, { retryPolicy: {maxAttempts: 5}})
    .addNode("query_database", queryDatabase, { retryPolicy: { retryOn: (e: any): boolean => {
        if (e instanceof Database.SqliteError) {
          // 出现“SQLITE_BUSY”错误时重试
          return e.code === 'SQLITE_BUSY';
        }
        return false; // Don't retry on other errors
      }}})
    .addEdge(START, "call_model")
    .addEdge("call_model", "query_database")
    .addEdge("query_database", END);

const graph = workflow.compile();